In [1]:
from happy_hub.models import OllamaModel

In [2]:
model = OllamaModel(
    url="http://34.1.198.45:11434/v1",
    model_config_dict={
        "model": "gemma2:27b",
        "temperature": 0.0,
        "stream": True
    }
)

In [17]:
model.run(messages=[
    {"role": "user", "content": "xin chào"}
])

In [4]:
import yaml # type: ignore

### Semantic Router

In [2]:
with open("../happy_hub/prompts/semantic_router.yml", 'r') as file:
    semantic_router_yaml = yaml.safe_load(file)

In [3]:
system_prompt_template = ""
for key in semantic_router_yaml:
    system_prompt_template += semantic_router_yaml[key]

In [5]:
from happy_hub.functions import RETRIEVAL_FUNCS

In [9]:
examples = [
    {"role": "user", "content": "xin chào"},
    {"role": "assistant",  'content': '{"name": "fun_chat"}'},
    {"role": "user", "content": "quy định nghỉ phép của MB gồm những gì?"},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'},
    {"role": "user", "content": "MB có gì hay ho nhỉ ?"},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'},
]

system_prompt = system_prompt_template.format(
    tools = [func.get_openai_tool_schema() for func in RETRIEVAL_FUNCS],
)

# Prepare the input messages
messages = [
    {"role": "user", "content": system_prompt},
    {"role": "assistant", "content": '{"name": "mb_info_query"}'}
]

messages.extend(examples)

messages.append({
    "role": "user", "content": "làm nhân viên chính thức có được giãm lãi vay vốn"
})

# Run the model with the input messages
response = model.run(messages=messages) # type: ignore

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

{"name": "mb_info_query"}

## Fun Chat

In [3]:
with open("../happy_hub/prompts/fun_chat.yml", 'r') as file:
    fun_chat_yaml = yaml.safe_load(file)

In [4]:
fun_chat_template = ""
for key in fun_chat_yaml:
    fun_chat_template += fun_chat_yaml[key]

In [5]:
# Prepare the input messages
fun_chat_messages = [
    {"role": "user", "content": fun_chat_template},
    {"role": "assistant", "content": 'Rất vui được giúp đỡ bạn, mình là happy hub!'}
]

fun_chat_messages.append({
    "role": "user", "content": "tôi muốn đi chơi hụi để giàu nhanh"
})

# Run the model with the input messages
response = model.run(messages=fun_chat_messages) # type: ignore

for chunk in response:
    print(chunk.choices[0].delta.content, end="")

NameError: name 'model' is not defined

## MB Info Query

In [5]:
with open("../happy_hub/prompts/mb_info_query.yml", 'r') as file:
    mb_info_query_yaml = yaml.safe_load(file)

mb_info_query_template = ""
for key in mb_info_query_yaml:
    mb_info_query_template += mb_info_query_yaml[key]

In [6]:
from happy_hub.storage import SimpleVectorDB

vector_db = SimpleVectorDB()
vector_db.load_from_file("../vector_db.pkl")

/home/gcp_dpf/happyhub-core/.venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [7]:
def parse_qa_text(text):
    """
    Parses a given text into a dictionary with 'question' and 'answer' keys.

    Args:
        text (str): The input text containing a question and answer separated by ';'.

    Returns:
        dict: A dictionary with 'question' and 'answer' as keys.

    Example:
        text = "question: Quy định về chế độ nghỉ phép có hưởng lương tại MB?; answer: Bạn có thời gian làm việc..."
        result = parse_qa_text(text)
    """
    # Split the text into question and answer parts
    parts = text.split(';')
    
    # Initialize the result dictionary
    result = {}
    
    # Parse the question
    if parts[0].startswith('question:'):
        result['question'] = parts[0].split('question:', 1)[1].strip()
    
    # Parse the answer
    if len(parts) > 1 and parts[1].startswith(' answer:'):
        result['answer'] = parts[1].split(' answer:', 1)[1].strip()
    
    return result

In [12]:
import json
results = vector_db.search("tín dụng xanh", top_k=1000)
document = []
for doc, score in results:
    print(parse_qa_text(doc))
    print('-'*30)

{'question': 'Đóng góp tiêu biểu của MB trong phát triển bền vững năm 2022', 'answer': '1. Kinh doanh: Lợi nhuận trước thuế hợp nhất: 22,729 tỷ đồng, Tổng vốn cho vay dự án xanh: 44,148 tỷ đồng (tăng 7% so với năm 2021).\n2. Môi trường: Gần một triệu khách hàng tham gia chiến dịch cùng MB phủ xanh Việt Nam , chung tay bằng cách đổi điểm MB star trên ứng dụng app MBBank.\n3. Nhân lực: MB ra mắt kênh học tập mới Nano series learning cho cán bộ nhân viên giúp MBers chủ động Học tập nhanh - Ứng dụng nhanh - Chuyển đổi nhanh (số giờ đào tạo bình quân năm 2022 là 35 giờ/ người/ năm)\n4. Cộng đồng: MB đóng góp 84 chương trình An sinh xã hội với 233 tỷ đồng. Dự án "Xây dựng mạng xã hội thiện nguyện Việt Nam" đã đạt giải A cuộc thi "Sáng kiến vì cộng đồng" lần thứ IV\n5. Quản trị:  MB có thành viên HĐQT chuyên trách các vấn đề về PTBV. Nỗ lực thực hiện các cam kết về thực hành quản trị công ty phù hợp quy định pháp luật Việt Nam hiện hành, tham chiếu và nghiên cứu áp dụng những thông lệ tốt tro

In [31]:
inject_documents = [
    """Đóng góp tiêu biểu của MB trong phát triển bền vững năm 2023;
1. Chuyển đổi số
Tiếp tục đẩy mạnh chuyển đổi số và chuyển dịch khách hàng lên kênh số:
 Tỷ lệ giao dịch trên kênh số đạt 96,7% tương đương 2,3 tỷ giao dịch, tăng trưởng 87% so với 2022 - khẳng định vị trí dẫn đầu về giao dịch toàn ngành ngân hàng và tương đương với tỷ lệ giao dịch kênh số của các ngân hàng Top đầu châu Á.
 Đưa ra thị trường các sản phẩm số giúp đẩy mạnh thanh toán không dùng tiền mặt tích hợp trên App MBBank và Biz MBBank như: Vòng tay thanh toán Stellar; Phát triển công cụ thanh toán VietQR tiện lợi, dễ dàng cho khách hàng doanh nghiệp.
 Chuyển tiền quốc tế online (cá nhân và doanh nghiệp) chuyển dịch 43%, tương đương 62 nghìn giao dịch, chuyển dịch giải ngân số đạt 50%, tương đương 233 nghìn giao dịch, thu nợ chuyển dịch 95%, tương đương 1,2 triệu giao dịch.
Triển khai Smartscan - số hóa hồ sơ:
 Chuyển dịch thẻ phi vật lý đạt 67%, tương đương 4,8 triệu thẻ phi vật lý, tiết kiệm chi phí in ấn và đóng gói thẻ.
 Triển khai Smartscan giúp lưu trữ 10 triệu file hồ sơ của khách hàng định danh, giảm 90% thời gian tìm kiếm và tra cứu hồ sơ, Chi phí vận hành và số lượng nhân sự vận hành/ khách hàng active giảm từ 25-30% so với 2022.
2. Tín dụng xanh MB tích cực phát triển các sản phẩm tín dụng xanh đa dạng, phù hợp, hiệu quả tới đối tượng khách hàng mục tiêu:
 Tổng dư cấp tín dụng xanh (bao gồm cho vay và trái phiếu) năm 2023 đạt 64.960 tỷ đồng, cho 3.762 khách hàng, đang trên đà tăng so với các năm trước và tăng 47% so với năm 2022.
 Xây dựng các trường dữ liệu để nhận diện được các khách hàng/ sản phẩm/ chương trình xanh trên hệ thống.
3. Môi trường
 Chiến dịch HiGreen – Bình minh xanh triển khai giai đoạn 1 - Thử thách chạy bộ, đã thu hút gần 30.000 người trên toàn quốc tham gia, hoàn thành 100% mục tiêu với 3 triệu km chạy bộ, 8,1 tỷ tiền tài trợ đối ứng trong năm 2023.
 Ra mắt thẻ HiGreen và kết nối với các đối tác theo tiêu chí có sứ mệnh, chiến lược bền vững, hoạt động tuân thủ tiêu chí ESG như MasterCard, Xanh SM, Coolmate, Cocoon. . .
 MB thay thế loại bóng chiếu sáng tiết kiệm 40% tại văn phòng.
 Rác thải nội bộ được phân loại ngay từ đầu nguồn tại điểm thu rác các tầng văn phòng bao gồm loại hữu cơ dễ phân hủy và vô cơ khó phân hủy
4. Con người MB
Các chương trình nổi bật hướng tới xây dựng văn hóa bền vững trong tổ chức bao gồm:
 Chương trình Flexi Z thu hút gần 1.700 nhân sự tập đoàn với mục tiêu đào tạo nhân sự 9x cho vị trí lãnh đạo cấp cao.
 Đào tạo bồi dưỡng nâng cao chất lượng nguồn lực thông qua chuỗi các chương trình đào tạo MB Leader, năng lực Digital, Bee Smart... Đặc biệt, các lớp đào tạo nhận thức về Phát triển bền vững được tổ chức trong năm 2023 cho hơn 120 cán bộ nhân viên đến từ khối Hội sở để đẩy mạnh hiệu quả triển khai tại các đơn vị.
 Xây dựng Bộ cẩm nang MB 1688 - Phương pháp làm việc của người MB giúp cán bộ nhân viên MB chuyển đổi tư duy số, trang bị các phương pháp số để thực thi công việc, phục vụ khách hàng theo một cách nhất quán với tầm nhìn, mục tiêu, định hướng của MB.
 Triển khai phong trào sáng kiến cải tiến hoạt động, sản phẩm dịch vụ (Beevoice), nâng cao tiếng nói của các MBer.
 Tổng số lượng học viên tham gia các khóa học trong năm 2023: 11.737 CBNV.
 Số giờ đào tạo bình quân năm 2023: 51,22 giờ (bao gồm cả đào tạo tân tuyển).
5. Cộng đồng
 Hơn 100 chương trình an sinh xã hội với tổng kinh phí đạt 247 tỷ đồng.
 MB đã có điểm giao dịch tại 05/10 tỉnh khó khăn nhất cả nước (Sơn La, Điện Biên, Gia Lai, Lào Cai, Yên Bái).
 Nền tảng xã hội thiện nguyện cung cấp tiện ích cho gần 1.200 tổ chức và cá nhân gây quỹ, tổ chức hơn 3.200 chiến dịch tính tới hết năm 2023
""",
    """Đóng góp tiêu biểu của MB trong phát triển bền vững năm 2022
1. Kinh doanh: Lợi nhuận trước thuế hợp nhất: 22,729 tỷ đồng, Tổng vốn cho vay dự án xanh: 44,148 tỷ đồng (tăng 7% so với năm 2021).
2. Môi trường: Gần một triệu khách hàng tham gia chiến dịch cùng MB phủ xanh Việt Nam , chung tay bằng cách đổi điểm MB star trên ứng dụng app MBBank.
3. Nhân lực: MB ra mắt kênh học tập mới Nano series learning cho cán bộ nhân viên giúp MBers chủ động Học tập nhanh - Ứng dụng nhanh - Chuyển đổi nhanh (số giờ đào tạo bình quân năm 2022 là 35 giờ/ người/ năm)
4. Cộng đồng: MB đóng góp 84 chương trình An sinh xã hội với 233 tỷ đồng. Dự án "Xây dựng mạng xã hội thiện nguyện Việt Nam" đã đạt giải A cuộc thi "Sáng kiến vì cộng đồng" lần thứ IV
5. Quản trị:  MB có thành viên HĐQT chuyên trách các vấn đề về PTBV. Nỗ lực thực hiện các cam kết về thực hành quản trị công ty phù hợp quy định pháp luật Việt Nam hiện hành, tham chiếu và nghiên cứu áp dụng những thông lệ tốt trong nước và quốc tế như Thông lệ tốt về QTCT của Việt Nam - VN CG Code và Nguyên tắc QTCT của OECD, v.v.""",    
"""Nguyên tắc quản trị của MB trong phát triển bền vững là gì
 Trách nhiệm với môi trường: quản trị và kiểm soát rủi ro môi trường – xã hội trong hoạt động cho vay và sau vay của ngân hàng, xác định các cơ hội liên quan tới môi trường trong hoạt động kinh doanh.
 Trách nhiệm với xã hội: thúc đẩy bình đẳng, tôn trọng các nguyên tắc quyền con người trong các chương trình và hoạt động kinh doanh của 2 ngân hàng. 
 Trách nhiệm với chính ngân hàng: nâng cao năng lực quản trị; tăng cường đào tạo, nâng cao chất lượng nguồn nhân lực; thúc đẩy văn hóa ngân hàng xanh tới toàn thể cán bộ, nhân viên ngân hàng; minh bạch thông tin trong nội bộ ngân hàng và các bên liên quan.
""",
"""Cam kết của MB để phát triển bền vững là gì?;
 MB quyết tâm chuyển đổi số mạnh mẽ. Sản phẩm, dịch vụ của MB hướng đến tăng trải nghiệm khách hàng, đề cao áp dụng tiêu chí ESG, góp phần thay đổi tích cực đến xã hội và kinh tế.
 Kinh doanh có trách nhiệm - hướng đến cung cấp các dịch vụ tín dụng và thanh toán trong các lĩnh vực thân thiện với môi trường.
 Đảm bảo nghĩa vụ tài chính, hỗ trợ thúc đẩy tăng trưởng kinh tế, đảm bảo kinh doanh liêm chính, nỗ lực thực hiện các cam kết với người lao động.
""",
"""Xếp hạng tín dụng của MB năm 2022;
Xếp hạng tín nhiệm MB từ B lên BB với triển vọng Tích cực góp phần khẳng định hoạt động an toàn, bền vững (theo Fitch Ratings)"""
]

In [32]:
def answer_mb_info_query(query: str):
    import json
    results = vector_db.search(query=query, top_k=5)
    documents = [doc for doc in inject_documents]
    # for doc, score in results:
    #     documents.append(str(parse_qa_text(doc)))
    system_prompt = mb_info_query_template.format(documents = '\n-----------------------------\n'.join(documents))
    
    # print(system_prompt)
    
    mb_query_messages = [
        {"role": "user", "content": system_prompt},
        {"role": "user", "content": query},
    ]
    response = model.run(mb_query_messages) # type: ignore
    for chunk in response:
        print(chunk.choices[0].delta.content, end="")

In [34]:
answer_mb_info_query("chiến dịch Hi Green")

Chiến dịch HiGreen – Bình minh xanh của MB đã triển khai giai đoạn 1 - Thử thách chạy bộ và thu hút gần 30.000 người trên toàn quốc tham gia. Chiến dịch đã hoàn thành 100% mục tiêu với 3 triệu km chạy bộ,  góp phần nâng cao nhận thức về lối sống xanh và bảo vệ môi trường.

Ngoài ra, MB còn triển khai thẻ tín dụng và tài khoản thanh toán mang tên HiGreen, một phần lợi nhuận từ các sản phẩm này sẽ được dùng để thực hiện các dự án vì môi trường. 
